### Imports and Helpers

In [2]:
import json, shlex, subprocess, re
from pathlib import Path

def run(cmd, check=True):
    """Run a shell command with pretty print."""
    print("$", cmd)
    return subprocess.run(shlex.split(cmd), check=check)

def find_repo_root(start: Path = None) -> Path:
    p = start or Path.cwd()
    for parent in [p, *p.resolve().parents]:
        if (parent / ".git").exists():
            return parent
    return Path.cwd()

def slugify(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[^a-z0-9]+", "-", s)
    s = re.sub(r"-+", "-", s).strip("-")
    return s[:60]

In [3]:
repo_root = find_repo_root()

# Ask user
doi = input("Enter Zenodo DOI (e.g., 10.5281/zenodo.16256961): ").strip()
if not doi:
    raise SystemExit("DOI is required.")


Enter Zenodo DOI (e.g., 10.5281/zenodo.16256961):  10.5281/zenodo.16256961


In [4]:
# Where to put downloaded files (in the repo)
default_dest = "data/raw"
dest_dir = input(f"Destination directory at repo root [{default_dest}]: ").strip() or default_dest

Destination directory at repo root [data/raw]:  


In [6]:
# Save config JSON for the workflow to read
metadata_dir = repo_root / "metadata"
metadata_dir.mkdir(parents=True, exist_ok=True)
cfg_path = metadata_dir / "zenodo_download.json"
cfg_path.write_text(json.dumps({"doi": doi, "dest_dir": dest_dir}, indent=2), encoding="utf-8")

print("✅ Saved config:", cfg_path.relative_to(repo_root))
print(cfg_path.read_text())

✅ Saved config: metadata/zenodo_download.json
{
  "doi": "10.5281/zenodo.16256961",
  "dest_dir": "data/raw"
}


In [7]:
# Commit this config so the workflow can read it on the runner
run("git add metadata/zenodo_download.json")
run('git commit -m "Add zenodo_download.json (DOI + destination for Zenodo download workflow)"', check=False)
run("git push origin main", check=False)

$ git add metadata/zenodo_download.json


fatal: pathspec 'metadata/zenodo_download.json' did not match any files


CalledProcessError: Command '['git', 'add', 'metadata/zenodo_download.json']' returned non-zero exit status 128.

In [ ]:
# Pull latest (avoid fast-forward issues)
run("git pull --rebase origin main", check=False)

tag = "zenodo-dl-" + slugify(doi)
# If tag exists already, append a counter suffix
existing = subprocess.run(shlex.split("git tag"), capture_output=True, text=True).stdout.splitlines()
if tag in existing:
    from datetime import datetime
    tag = f"{tag}-{datetime.utcnow().strftime('%H%M%S')}"

run(f"git tag {tag}")
run("git push origin --tags")
print(f"✅ Triggered workflow with tag: {tag}")
print("➡️  Go to GitHub → Actions → watch the 'Zenodo Download' run. Files will be committed to the repo in your chosen folder.")